In [0]:
!pip install fast-bert
!pip install torch==1.3.0+cu100 torchvision==0.4.2+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install datetime
!pip install fastai
!pip install pytorch-transformers
!pip install python-box
!pip install torch torchvision

     |████████████████████████████████| 71kB 3.1MB/s 
     |████████████████████████████████| 389kB 7.8MB/s 
     |████████████████████████████████| 194kB 11.4MB/s 
     |████████████████████████████████| 860kB 12.2MB/s 
     |████████████████████████████████| 696kB 14.1MB/s 
     |████████████████████████████████| 1.0MB 19.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=cfccadf6c0343995ce4151835069f29017126766c8a918a882c621b9ab41eaf2
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 744.5MB 23kB/s 
     |████████████████████████████████| 10.2MB 88.3MB/s 
ERROR: torchvision 0.4.2+cu100 has requirement torch==1.3.1, but you'll have torch 1.3.0+cu100 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3

In [0]:
#@title
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ..


Cloning into 'apex'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 5603 (delta 2), reused 8 (delta 2), pack-reused 5585
Receiving objects: 100% (5603/5603), 13.36 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (3623/3623), done.
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-gjq6opee
Created temporary directory: /tmp/pip-req-tracker-kn95_sih
Created requirements tracker '/tmp/pip-req-tracker-kn95_sih'
Created temporary directory: /tmp/pip-install-ng_3am8t
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-48o15rxr
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-kn95_sih'
    Runn

In [0]:
!git clone https://github.com/ttpss930141011/Project.git

Cloning into 'Project'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 307 (delta 15), reused 20 (delta 9), pack-reused 278
Receiving objects: 100% (307/307), 57.11 MiB | 23.21 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [0]:
from box import Box
import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')
if not LOG_PATH.exists():
  LOG_PATH.mkdir()
MODEL_PATH=Path('models/')
if not MODEL_PATH.exists():
  MODEL_PATH.mkdir()
DATA_PATH = Path('Project/sample/data/')     
LABEL_PATH = Path('Project/sample/labels/')   
BERT_PRETRAINED_PATH = "bert-base-chinese"

OUTPUT_PATH=Path('output/')
if not OUTPUT_PATH.exists():
  OUTPUT_PATH.mkdir()
args = Box({
    "run_text": "multilabel toxic comments with freezable layers",
    "train_size": -1,
    "val_size": -1,
    "log_path": LOG_PATH,
    "full_data_dir": DATA_PATH,
    "data_dir": DATA_PATH,
    "task_name": "toxic_classification_lib",
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "max_seq_length": 512,
    "output_dir": OUTPUT_PATH,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 8,
    "eval_batch_size": 16,
    "learning_rate": 5e-5,
    "num_train_epochs": 6,
    "warmup_proportion": 0.0,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": True,
    "fp16_opt_level": "O1",
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "max_grad_norm": 1.0,
    "max_steps": -1,
    "warmup_steps": 500,
    "logging_steps": 50,
    "eval_all_checkpoints": True,
    "overwrite_output_dir": True,
    "overwrite_cache": False,
    "seed": 42,
    "loss_scale": 128,
    "task_name": 'intent'
})


import logging

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

torch.cuda.empty_cache() 


In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
import torch

torch.version.cuda

'10.0.130'

In [0]:
torch.cuda.is_available()

True

In [0]:
from pytorch_transformers import BertTokenizer
from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc


12/19/2019 11:11:37 - INFO - transformers.file_utils -   PyTorch version 1.3.0+cu100 available.


In [0]:
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, 
                                          do_lower_case=args['do_lower_case'])

12/19/2019 11:11:37 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmp5lnkv16f


100%|██████████| 109540/109540 [00:00<00:00, 876000.87B/s]

12/19/2019 11:11:37 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp5lnkv16f to cache at /root/.cache/torch/pytorch_transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/19/2019 11:11:37 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/19/2019 11:11:37 - INFO - pytorch_transformers.file_utils -   removing temp file /tmp/tmp5lnkv16f
12/19/2019 11:11:37 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [0]:
label_cols = ["label1", "label2", "label3" ]

In [0]:
databunch = BertDataBunch(DATA_PATH,
              LABEL_PATH,
              tokenizer = 'bert-base-chinese', 
              train_file = 'train.csv', 
              val_file = 'valid.csv',
              test_data = 'test.csv',
              text_col = "text", 
              label_col = label_cols,
              batch_size_per_gpu = 8, 
              max_seq_length = 512, 
              multi_gpu = multi_gpu, 
              multi_label = True,
              model_type ='bert')

12/19/2019 11:11:38 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmp1bxzgcyi



12/19/2019 11:11:38 - INFO - transformers.file_utils -   copying /tmp/tmp1bxzgcyi to cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/19/2019 11:11:38 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/19/2019 11:11:38 - INFO - transformers.file_utils -   removing temp file /tmp/tmp1bxzgcyi
12/19/2019 11:11:38 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
12/19/2019 11:11:39 - INFO - root -   Writing example 0 of 1849
12/19/

In [0]:
logger.info(args)

12/19/2019 11:11:41 - INFO - root -   {'run_text': 'multilabel toxic comments with freezable layers', 'train_size': -1, 'val_size': -1, 'log_path': PosixPath('logs'), 'full_data_dir': PosixPath('Project/sample/data'), 'data_dir': PosixPath('Project/sample/data'), 'task_name': 'intent', 'no_cuda': False, 'bert_model': 'bert-base-chinese', 'max_seq_length': 512, 'output_dir': PosixPath('output'), 'do_train': True, 'do_eval': True, 'do_lower_case': True, 'train_batch_size': 8, 'eval_batch_size': 16, 'learning_rate': 5e-05, 'num_train_epochs': 6, 'warmup_proportion': 0.0, 'local_rank': -1, 'seed': 42, 'gradient_accumulation_steps': 1, 'optimize_on_cpu': False, 'fp16': True, 'fp16_opt_level': 'O1', 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'max_steps': -1, 'warmup_steps': 500, 'logging_steps': 50, 'eval_all_checkpoints': True, 'overwrite_output_dir': True, 'overwrite_cache': False, 'loss_scale': 128}


In [0]:
metrics = []

metrics.append({'name': 'accuracy', 'function': accuracy_multilabel})
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})

In [0]:

learner = BertLearner.from_pretrained_model(databunch,
                                            "bert-base-chinese",
                                            metrics=metrics, 
                                            device=device,
                                            logger=logger,
                                            warmup_steps=500,
                                            multi_gpu=multi_gpu,
                                            is_fp16=True, 
                                            multi_label=True,
                                            logging_steps=0,
                                            output_dir=OUTPUT_PATH)

12/19/2019 11:11:41 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json not found in cache or force_download set to True, downloading to /tmp/tmp736pxzz_



12/19/2019 11:11:42 - INFO - transformers.file_utils -   copying /tmp/tmp736pxzz_ to cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
12/19/2019 11:11:42 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
12/19/2019 11:11:42 - INFO - transformers.file_utils -   removing temp file /tmp/tmp736pxzz_
12/19/2019 11:11:42 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
12/19/2019 11:11:42 - INFO - transformers.configurati


12/19/2019 11:11:54 - INFO - transformers.file_utils -   copying /tmp/tmp3flqie6q to cache at /root/.cache/torch/transformers/b1b5e295889f2d0979ede9a78ad9cb5dc6a0e25ab7f9417b315f0a2c22f4683d.929717ca66a3ba9eb9ec2f85973c6398c54c38a4faa464636a491d7a705f7eb6
12/19/2019 11:11:55 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/b1b5e295889f2d0979ede9a78ad9cb5dc6a0e25ab7f9417b315f0a2c22f4683d.929717ca66a3ba9eb9ec2f85973c6398c54c38a4faa464636a491d7a705f7eb6
12/19/2019 11:11:55 - INFO - transformers.file_utils -   removing temp file /tmp/tmp3flqie6q
12/19/2019 11:11:55 - INFO - transformers.modeling_utils -   loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-pytorch_model.bin from cache at /root/.cache/torch/transformers/b1b5e295889f2d0979ede9a78ad9cb5dc6a0e25ab7f9417b315f0a2c22f4683d.929717ca66a3ba9eb9ec2f85973c6398c54c38a4faa464636a491d7a705f7eb6
12/19/2019 11:11:58 - INFO - transformers.modeling_utils -

In [0]:

learner.fit(4, lr=5e-5,schedule_type="warmup_linear" )

output/tensorboard
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
12/19/2019 11:12:03 - INFO - root -   ***** Running training *****
12/19/2019 11:12:03 - INFO - root -     Num examples = 1849
12/19/2019 11:12:03 - INFO - root -     Num Epochs = 4
12/19/2019 11:12:03 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 8
12/

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
12/19/2019 11:14:08 - INFO - root -   Running evaluation
12/19/2019 11:14:08 - INFO - root -     Num examples = 391
12/19/2019 11:14:08 - INFO - root -     Batch size = 16


12/19/2019 11:14:15 - INFO - root -   eval_loss after epoch 1: 0.43299936652183535: 
12/19/2019 11:14:15 - INFO - root -   eval_accuracy after epoch 1: 0.5421994884910486: 
12/19/2019 11:14:15 - INFO - root -   eval_accuracy_thresh after epoch 1: 0.9386189579963684: 
12/19/2019 11:14:15 - INFO - root -   eval_roc_auc after epoch 1: 0.6576071675059431: 
12/19/2019 11:14:15 - INFO - root -   eval_fbeta after epoch 1: 0.12148337811231613: 
12/19/2019 11:14:15 - INFO - root -   lr after epoch 1: 2.32e-05
12/19/2019 11:14:15 - INFO - root -   train_loss after epoch 1: 0.5226278042998808
12/19/2019 11:14:15 - INFO - root -   

12/19/2019 11:16:21 - INFO - root -   Running evaluation
12/19/2019 11:16:21 - INFO - root -     Num examples = 391
12/19/2019 11:16:21 - INFO - root -     Batch size = 16


12/19/2019 11:16:28 - INFO - root -   eval_loss after epoch 2: 0.2526805502176285: 
12/19/2019 11:16:28 - INFO - root -   eval_accuracy after epoch 2: 0.6061381074168798: 
12/19/2019 11:16:28 - INFO - root -   eval_accuracy_thresh after epoch 2: 0.9394714832305908: 
12/19/2019 11:16:28 - INFO - root -   eval_roc_auc after epoch 2: 0.7888154699521996: 
12/19/2019 11:16:28 - INFO - root -   eval_fbeta after epoch 2: 0.0025575447361916304: 
12/19/2019 11:16:28 - INFO - root -   lr after epoch 2: 4.64e-05
12/19/2019 11:16:28 - INFO - root -   train_loss after epoch 2: 0.32247297170347183
12/19/2019 11:16:28 - INFO - root -   

12/19/2019 11:18:33 - INFO - root -   Running evaluation
12/19/2019 11:18:33 - INFO - root -     Num examples = 391
12/19/2019 11:18:33 - INFO - root -     Batch size = 16


12/19/2019 11:18:41 - INFO - root -   eval_loss after epoch 3: 0.19404789537191391: 
12/19/2019 11:18:41 - INFO - root -   eval_accuracy after epoch 3: 0.5703324808184144: 
12/19/2019 11:18:41 - INFO - root -   eval_accuracy_thresh after epoch 3: 0.9394714832305908: 
12/19/2019 11:18:41 - INFO - root -   eval_roc_auc after epoch 3: 0.8963216686690013: 
12/19/2019 11:18:41 - INFO - root -   eval_fbeta after epoch 3: 0.0: 
12/19/2019 11:18:41 - INFO - root -   lr after epoch 3: 2.7102803738317755e-05
12/19/2019 11:18:41 - INFO - root -   train_loss after epoch 3: 0.19899344365594202
12/19/2019 11:18:41 - INFO - root -   

12/19/2019 11:20:46 - INFO - root -   Running evaluation
12/19/2019 11:20:46 - INFO - root -     Num examples = 391
12/19/2019 11:20:46 - INFO - root -     Batch size = 16


12/19/2019 11:20:53 - INFO - root -   eval_loss after epoch 4: 0.17935007363557814: 
12/19/2019 11:20:53 - INFO - root -   eval_accuracy after epoch 4: 0.7442455242966752: 
12/19/2019 11:20:53 - INFO - root -   eval_accuracy_thresh after epoch 4: 0.9394714832305908: 
12/19/2019 11:20:53 - INFO - root -   eval_roc_auc after epoch 4: 0.9172183737634518: 
12/19/2019 11:20:53 - INFO - root -   eval_fbeta after epoch 4: 0.005115089472383261: 
12/19/2019 11:20:53 - INFO - root -   lr after epoch 4: 0.0
12/19/2019 11:20:53 - INFO - root -   train_loss after epoch 4: 0.16723252466783442
12/19/2019 11:20:53 - INFO - root -   



(928, 0.30283168608178224)

In [0]:
learner.validate()

12/19/2019 11:20:53 - INFO - root -   Running evaluation
12/19/2019 11:20:53 - INFO - root -     Num examples = 391
12/19/2019 11:20:53 - INFO - root -     Batch size = 16


{'accuracy': 0.7442455242966752,
 'accuracy_thresh': 0.9394714832305908,
 'fbeta': 0.005115089472383261,
 'loss': 0.17935007363557814,
 'roc_auc': 0.9172183737634518}

In [0]:
learner.save_model()

12/19/2019 11:21:01 - INFO - transformers.configuration_utils -   Configuration saved in output/model_out/config.json
12/19/2019 11:21:02 - INFO - transformers.modeling_utils -   Model weights saved in output/model_out/pytorch_model.bin


In [0]:

texts1 = [
  "我想吃東西",
  
] 
texts2 = [

  "展現 真正 的 自己 很 重要 "
]

predictions1 = learner.predict_batch(texts1)
predictions2 = learner.predict_batch(texts2)

12/19/2019 11:21:02 - INFO - root -   Writing example 0 of 1
12/19/2019 11:21:02 - INFO - root -   Writing example 0 of 1


In [0]:
predictions1


[[('label1', 0.10931396484375),
  ('label2', 0.06036376953125),
  ('label3', 0.0357666015625)]]

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

12/19/2019 11:22:20 - INFO - numexpr.utils -   NumExpr defaulting to 2 threads.


KeyboardInterrupt: ignored

In [0]:
predictions2

[[('label2', 0.227294921875),
  ('label3', 0.123046875),
  ('label1', 0.11376953125)]]